# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import numpy as np
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report,fbeta_score,make_scorer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
import pickle
nltk.download(['punkt', 'wordnet','stopwords'])







[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = df = pd.read_sql('SELECT * FROM disaster_response', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

Message column is the input and we have to classify what kind of message it is 
so X is column message
and Y is columns from related to direct_report, so we are droping id, message, original, genre columns for y

In [5]:
df['search_and_rescue'].unique()

array([0, 1])

In [6]:
X = df['message']

In [7]:

Y = df.drop(['id','message','original','genre'],axis=1)

In [8]:
Y.dtypes

related                   int64
request                   int64
offer                     int64
aid_related               int64
medical_help              int64
medical_products          int64
search_and_rescue         int64
security                  int64
military                  int64
child_alone               int64
water                     int64
food                      int64
shelter                   int64
clothing                  int64
money                     int64
missing_people            int64
refugees                  int64
death                     int64
other_aid                 int64
infrastructure_related    int64
transport                 int64
buildings                 int64
electricity               int64
tools                     int64
hospitals                 int64
shops                     int64
aid_centers               int64
other_infrastructure      int64
weather_related           int64
floods                    int64
storm                     int64
fire    

In [9]:
column_names=Y.columns

In [10]:
column_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [11]:
for col in column_names:
    print(df[col].unique())

[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


### 2. Write a tokenization function to process your text data

In [12]:


clean_tokens=[]
for message in X[:10]:
    text=message.lower()
    text=re.sub(r"[^a-zA-Z0-9]"," ",text)
    tokens = word_tokenize(text)
    
lemmatizer=WordNetLemmatizer()
stop_word = stopwords.words("english")
for toks in tokens:
    if toks not in stop_word:
        clean_tok=lemmatizer.lemmatize(toks).strip()  
        clean_tokens.append(clean_tok)

print(clean_tokens)


['croix', 'de', 'bouquet', 'health', 'issue', 'worker', 'santo', '15', 'area', 'croix', 'de', 'bouquet']


In [13]:
def tokenize(text):
    """
    " load data from database
    "
    " Args:
    "   text: the text to be tokenized
    "
    " Returns:
    "   tokens: the tokens extracted from the text
    "
    """
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()).strip()

    # tokenize text
    tokens = word_tokenize(text)

    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
pipeline = Pipeline([("vect",CountVectorizer(tokenizer=tokenize)),
                    ("tfidf",TfidfTransformer()),
                    ("clf",MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state = 42)


In [16]:
np.random.seed(42)

In [17]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
y_pred_train=pipeline.predict(X_train)

In [19]:
# from sklearn.metrics import classification_report
#y_preds, Y_test.values

print(classification_report(y_train,y_pred_train, target_names= column_names))

                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     16095
               request       1.00      0.93      0.96      3583
                 offer       1.00      0.67      0.80        94
           aid_related       1.00      0.97      0.98      8696
          medical_help       1.00      0.86      0.92      1649
      medical_products       1.00      0.85      0.92      1034
     search_and_rescue       1.00      0.79      0.88       588
              security       1.00      0.72      0.84       375
              military       1.00      0.86      0.93       702
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.94      0.97      1337
                  food       1.00      0.95      0.98      2339
               shelter       1.00      0.91      0.95      1846
              clothing       1.00      0.85      0.92       335
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [20]:
y_pred_test = pipeline.predict(X_test)

In [21]:
print(classification_report(y_test, y_pred_test, target_names=column_names))


                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      3998
               request       0.79      0.44      0.57       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.74      0.58      0.65      2164
          medical_help       0.48      0.07      0.12       435
      medical_products       0.69      0.10      0.18       279
     search_and_rescue       0.50      0.07      0.13       136
              security       0.00      0.00      0.00        96
              military       0.52      0.08      0.13       158
           child_alone       0.00      0.00      0.00         0
                 water       0.81      0.31      0.45       335
                  food       0.87      0.43      0.57       584
               shelter       0.79      0.36      0.50       468
              clothing       0.56      0.07      0.13        70
                 money       0.57      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f5f6982a048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [23]:
parameters = {   
                  'tfidf__use_idf': [True]                 
                 
              }

In [24]:
cv = GridSearchCV(pipeline,param_grid=parameters, verbose = 10)

In [25]:
model=cv.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] tfidf__use_idf=True .............................................
[CV] ... tfidf__use_idf=True, score=0.24274066657130597, total= 2.7min
[CV] tfidf__use_idf=True .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.0min remaining:    0.0s


[CV] ... tfidf__use_idf=True, score=0.24173937920183092, total= 2.6min
[CV] tfidf__use_idf=True .............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.8min remaining:    0.0s


[CV] ... tfidf__use_idf=True, score=0.24334763948497853, total= 2.6min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 11.7min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 11.7min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
y_pred_train = cv.predict(X_train)
print(classification_report(y_train.values, y_pred_train, target_names=column_names))

                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     16095
               request       1.00      0.93      0.96      3583
                 offer       1.00      0.69      0.82        94
           aid_related       0.99      0.97      0.98      8696
          medical_help       1.00      0.85      0.92      1649
      medical_products       1.00      0.84      0.91      1034
     search_and_rescue       1.00      0.79      0.88       588
              security       1.00      0.75      0.86       375
              military       1.00      0.88      0.94       702
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.93      0.97      1337
                  food       1.00      0.95      0.97      2339
               shelter       1.00      0.92      0.96      1846
              clothing       1.00      0.88      0.94       335
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [27]:
y_pred_test = cv.predict(X_test)
print(classification_report(y_test.values, y_pred_test, target_names=column_names))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      3998
               request       0.79      0.43      0.56       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.77      0.60      0.67      2164
          medical_help       0.63      0.13      0.21       435
      medical_products       0.67      0.12      0.20       279
     search_and_rescue       0.56      0.07      0.13       136
              security       0.00      0.00      0.00        96
              military       0.47      0.06      0.10       158
           child_alone       0.00      0.00      0.00         0
                 water       0.80      0.29      0.42       335
                  food       0.84      0.48      0.61       584
               shelter       0.79      0.33      0.47       468
              clothing       0.92      0.17      0.29        70
                 money       0.77      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [28]:
cv.best_params_

{'tfidf__use_idf': True}

In [29]:
# Try using AdaBoost instead of Random Forest Classifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True]
              }

cv2 = GridSearchCV(pipeline2, param_grid = parameters2,  verbose = 10)


In [30]:

# Find best parameters
np.random.seed(42)
model2 = cv2.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] tfidf__use_idf=True, vect__min_df=5 .............................
[CV]  tfidf__use_idf=True, vect__min_df=5, score=0.24159633814904877, total= 2.8min
[CV] tfidf__use_idf=True, vect__min_df=5 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.0min remaining:    0.0s


[CV]  tfidf__use_idf=True, vect__min_df=5, score=0.23415820340437707, total= 2.8min
[CV] tfidf__use_idf=True, vect__min_df=5 .............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  8.1min remaining:    0.0s


[CV]  tfidf__use_idf=True, vect__min_df=5, score=0.24763948497854077, total= 2.9min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 12.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 12.2min finished


### 9. Export your model as a pickle file

In [31]:
pickle.dump(model, open("disaster_Response_model.pkl",'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.